In [1]:
import requests
import pandas as pd
import pprint as pp

In [2]:
def advanced_company_search(**kwargs):
    api_key = "9ca44c28-e54d-4c26-9bf7-6c766c53c054"
    url = "https://api.company-information.service.gov.uk/advanced-search/companies"
    headers = {"Authorization": api_key}
    response = requests.get(url, headers=headers, params=kwargs)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": "Request failed", "status_code": response.status_code}


# 示例用法
search_params = {
    "sic_codes": "64191",
    "incorporated_from": "2015-01-01",
    "incorporated_to": "2023-01-01",
    "size": 5000,
    "start_index": 0,
}
results_original = advanced_company_search(**search_params)

In [3]:
results = results_original["items"]

print(len(results))

locations = []

2760


In [4]:
# Convert the list to a DataFrame
df = pd.DataFrame(results)


In [5]:
# Expand the 'registered_office_address' column into separate columns
address_df = df["registered_office_address"].apply(pd.Series)

# Join the new columns with the original DataFrame
df = pd.concat([df.drop(["registered_office_address"], axis=1), address_df], axis=1)

# Convert 'date_of_cessation' and 'date_of_creation' to datetime and get the year
df["year_of_cessation"] = pd.to_datetime(df["date_of_cessation"]).dt.year
df["year_of_creation"] = pd.to_datetime(df["date_of_creation"]).dt.year
df['year_of_cessation'] = df['year_of_cessation'].astype('Int64')

# Display the first few rows of the DataFrame
df.head()

,company_name,company_number,company_status,company_type,kind,links,date_of_cessation,date_of_creation,sic_codes,company_subtype,address_line_1,locality,postal_code,country,region,address_line_2,year_of_cessation,year_of_creation
0,CAAMAQ LTD.,09638422,dissolved,ltd,search-results#company,{'company_profile': '/company/09638422'},2017-11-21,2015-06-15,[64191],NaN,107 Penarth Road Penarth Road,Cardiff,CF11 6JT,Wales,NaN,NaN,2017,2015
1,XELANT CONSULTING LTD,09638749,dissolved,ltd,search-results#company,{'company_profile': '/company/09638749'},2018-03-06,2015-06-15,[64191],NaN,15 Braybrooke Gardens,Saffron Walden,CB11 3WH,England,Essex,NaN,2018,2015
2,KJLIU LIMITED,09643634,dissolved,ltd,search-results#company,{'company_profile': '/company/09643634'},2023-08-22,2015-06-17,[64191],NaN,9 Augustine Drive,Ashford,TN25 7GH,England,NaN,Finberry,2023,2015
3,FRLC SERVICES LIMITED,09646354,dissolved,ltd,search-results#company,{'company_profile': '/company/09646354'},2021-05-09,2015-06-18,[64191],NaN,41 Kingston Street,Cambridge,CB1 2NU,NaN,NaN,NaN,2021,2015
4,SANAM SOLUTIONS LTD,09659664,active,ltd,search-results#company,{'company_profile': '/company/09659664'},NaN,2015-06-26,[64191],NaN,410 Gillott Road,Birmingham,B16 9LP,England,NaN,NaN,<NA>,2015


In [6]:
# Select rows where 'locality' is 'Birmingham' and 'year_of_creation' is 2015
selected_rows = df[(df['locality'] == 'Bradford') & (df['year_of_creation'] == 2017)]

# Count the number of such rows
num_companies = len(selected_rows)

# Display the result
num_companies

24

In [7]:
def calculate_cessation_rate(df):
    # 初始化一个新的DataFrame来存储结果
    years = range(2015, 2023)
    columns = ['locality'] + [f'{year}_created' for year in years] + [f'{year}_ceased' for year in years] + [f'{year}_cessation_rate' for year in years]
    summary_df = pd.DataFrame(columns=columns)
    summary_df['locality'] = df['locality'].unique()

    # 遍历每年进行计算
    for year in years:
        # 计算每个城市在特定年份创建的公司数量
        created = df[df['year_of_creation'] == year].groupby('locality')['company_number'].count()
        # 计算每个城市在特定年份停业的公司数量
        ceased = df[df['year_of_cessation'] == year].groupby('locality')['company_number'].count()

        summary_df[f'{year}_created'] = summary_df['locality'].map(created)
        summary_df[f'{year}_ceased'] = summary_df['locality'].map(ceased)
        summary_df[f'{year}_cessation_rate'] = summary_df[f'{year}_ceased'] / summary_df[f'{year}_created']

    # 处理NaN值
    summary_df.fillna(0, inplace=True)
    
    return summary_df

# 使用函数计算并生成新的DataFrame
new_df = calculate_cessation_rate(df)
new_df.head()  # 展示前几行以检查结果

,locality,2015_created,2016_created,2017_created,2018_created,2019_created,2020_created,2021_created,2022_created,2015_ceased,...,2021_ceased,2022_ceased,2015_cessation_rate,2016_cessation_rate,2017_cessation_rate,2018_cessation_rate,2019_cessation_rate,2020_cessation_rate,2021_cessation_rate,2022_cessation_rate
0,Cardiff,19.0,4.0,16.0,16.0,19.0,9.0,6.0,13.0,0.0,...,22.0,9.0,0.0,0.25,0.500000,0.250000,0.842105,1.222222,3.666667,0.692308
1,Saffron Walden,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Ashford,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Cambridge,6.0,1.0,8.0,3.0,1.0,0.0,0.0,1.0,0.0,...,8.0,7.0,0.0,0.00,0.000000,0.333333,2.000000,0.000000,0.000000,7.000000
4,Birmingham,9.0,4.0,11.0,19.0,13.0,3.0,0.0,1.0,0.0,...,10.0,6.0,0.0,0.00,0.272727,0.105263,0.384615,3.666667,0.000000,6.000000


In [8]:
new_df.loc[new_df['locality'] == 'Cardiff', '2015_ceased']

0    0.0
Name: 2015_ceased, dtype: float64

In [9]:
# 计算每个城市的公司数量
counts = df["locality"].value_counts()

# 获取公司数量大于30的城市列表
counts_greater_than30 = counts[counts > 30]
cities_to_keep = counts_greater_than30.index.tolist()

# 过滤数据框以保留这些城市的数据
filtered_df = new_df[new_df["locality"].isin(cities_to_keep)]

In [10]:
filtered_df

,locality,2015_created,2016_created,2017_created,2018_created,2019_created,2020_created,2021_created,2022_created,2015_ceased,...,2021_ceased,2022_ceased,2015_cessation_rate,2016_cessation_rate,2017_cessation_rate,2018_cessation_rate,2019_cessation_rate,2020_cessation_rate,2021_cessation_rate,2022_cessation_rate
0,Cardiff,19.0,4.0,16.0,16.0,19.0,9.0,6.0,13.0,0.0,...,22.0,9.0,0.0,0.250000,0.500000,0.250000,0.842105,1.222222,3.666667,0.692308
4,Birmingham,9.0,4.0,11.0,19.0,13.0,3.0,0.0,1.0,0.0,...,10.0,6.0,0.0,0.000000,0.272727,0.105263,0.384615,3.666667,0.000000,6.000000
5,London,73.0,76.0,146.0,133.0,104.0,133.0,143.0,124.0,0.0,...,141.0,136.0,0.0,0.052632,0.130137,0.473684,0.961538,0.601504,0.986014,1.096774
20,Manchester,5.0,7.0,11.0,16.0,5.0,2.0,4.0,2.0,0.0,...,9.0,4.0,0.0,0.000000,0.090909,0.312500,1.400000,2.000000,2.250000,2.000000
25,Liverpool,14.0,12.0,15.0,15.0,11.0,2.0,2.0,2.0,0.0,...,18.0,5.0,0.0,0.083333,0.133333,0.400000,0.909091,8.500000,9.000000,2.500000
35,Leeds,8.0,1.0,20.0,8.0,8.0,0.0,0.0,0.0,0.0,...,11.0,7.0,0.0,0.000000,0.150000,0.250000,0.875000,0.000000,0.000000,0.000000
45,Bradford,7.0,2.0,24.0,12.0,14.0,2.0,2.0,0.0,0.0,...,14.0,6.0,0.0,0.000000,0.125000,0.250000,0.571429,6.000000,7.000000,0.000000
118,Glasgow,11.0,6.0,18.0,11.0,15.0,3.0,0.0,0.0,0.0,...,19.0,6.0,0.0,0.000000,0.000000,0.636364,0.400000,3.333333,0.000000,0.000000
122,Edinburgh,6.0,6.0,11.0,6.0,3.0,0.0,2.0,0.0,0.0,...,6.0,3.0,0.0,0.000000,0.000000,0.666667,2.000000,0.000000,3.000000,0.000000
147,Leicester,5.0,2.0,7.0,7.0,13.0,3.0,2.0,1.0,0.0,...,9.0,5.0,0.0,0.000000,0.142857,0.142857,0.384615,2.000000,4.500000,5.000000


In [11]:
from geopy.geocoders import Nominatim
import pandas as pd

# 确保您已加载了DataFrame 'filtered_df'
# 如果还没有加载，可以使用类似下面的代码加载它：
# filtered_df = pd.read_csv('path_to_your_csv_file.csv')  # 替换为您的CSV文件路径

# 初始化地理编码器
geolocator = Nominatim(user_agent="geoapiExercises")

# 函数：根据城市名获取纬度和经度
def get_lat_long(city):
    try:
        location = geolocator.geocode(city + ", UK")  # 假设所有城市都在英国
        return location.latitude, location.longitude
    except:
        return None, None

# 为数据集中的每个城市添加经纬度
for city in filtered_df['locality'].unique():
    lat, lon = get_lat_long(city)
    filtered_df.loc[filtered_df['locality'] == city, 'latitude'] = lat
    filtered_df.loc[filtered_df['locality'] == city, 'longitude'] = lon

# 检查添加了坐标的数据集
filtered_df.head()
# 可以选择保存更新后的数据


C:\Users\redhawc\AppData\Local\Temp\ipykernel_21056\3311726706.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[filtered_df['locality'] == city, 'latitude'] = lat
C:\Users\redhawc\AppData\Local\Temp\ipykernel_21056\3311726706.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[filtered_df['locality'] == city, 'longitude'] = lon


,locality,2015_created,2016_created,2017_created,2018_created,2019_created,2020_created,2021_created,2022_created,2015_ceased,...,2015_cessation_rate,2016_cessation_rate,2017_cessation_rate,2018_cessation_rate,2019_cessation_rate,2020_cessation_rate,2021_cessation_rate,2022_cessation_rate,latitude,longitude
0,Cardiff,19.0,4.0,16.0,16.0,19.0,9.0,6.0,13.0,0.0,...,0.0,0.250000,0.500000,0.250000,0.842105,1.222222,3.666667,0.692308,51.481655,-3.179193
4,Birmingham,9.0,4.0,11.0,19.0,13.0,3.0,0.0,1.0,0.0,...,0.0,0.000000,0.272727,0.105263,0.384615,3.666667,0.000000,6.000000,52.479699,-1.902691
5,London,73.0,76.0,146.0,133.0,104.0,133.0,143.0,124.0,0.0,...,0.0,0.052632,0.130137,0.473684,0.961538,0.601504,0.986014,1.096774,51.507446,-0.127765
20,Manchester,5.0,7.0,11.0,16.0,5.0,2.0,4.0,2.0,0.0,...,0.0,0.000000,0.090909,0.312500,1.400000,2.000000,2.250000,2.000000,53.479489,-2.245115
25,Liverpool,14.0,12.0,15.0,15.0,11.0,2.0,2.0,2.0,0.0,...,0.0,0.083333,0.133333,0.400000,0.909091,8.500000,9.000000,2.500000,53.407199,-2.991680


In [14]:
import plotly.express as px
import pandas as pd

# 假设您已经加载了带有经纬度的filtered_df
# filtered_df = pd.read_csv('path_to_your_updated_csv_file.csv')

# 准备数据，将年份数据合并到一列
data = []
for year in range(2016, 2023):
    temp_df = filtered_df[
        [
            "locality",
            "latitude",
            "longitude",
            f"{year}_created",
            f"{year}_cessation_rate",
        ]
    ]
    temp_df["year"] = year
    temp_df = temp_df.rename(
        columns={
            f"{year}_created": "Bank Created",
            f"{year}_cessation_rate": "Cessation Rate",
        }
    )
    data.append(temp_df)
map_data = pd.concat(data)

# 确定cessation_rate的最小值和最大值
min_cessation_rate = map_data["Cessation Rate"].min()
max_cessation_rate = map_data["Cessation Rate"].max()

custom_color_scale = [[0, "yellow"], [1, "red"]]  # 浅黄色  # 深红色

# 创建气泡图
fig = px.scatter_mapbox(
    map_data,
    lat="latitude",
    lon="longitude",
    hover_name="locality",
    hover_data=["Bank Created", "Cessation Rate"],
    color="Cessation Rate",
    size="Bank Created",
    animation_frame="year",
    color_continuous_scale=custom_color_scale,
    range_color=[min_cessation_rate, max_cessation_rate],  # 设置颜色比例尺范围
    size_max=100,
    zoom=4.9,
    height=600,
)

# 设置地图样式
fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r": 0, "t": 50, "l": 0, "b": 0},
    title="Visualisation of New Bank Creation and Bank Cessation Rate<br> in the UK from 2016 to 2022",
)

fig.update_traces(marker=dict(opacity=0.80))
# 添加注释
fig.add_annotation(
    text="The larger the bubble size, the more banks are established in the area;<br> the darker the color, the higher the bank closure rate (= number of closed banks / number of new banks) in the area.",
    align="left",
    showarrow=False,
    xref="paper",
    yref="paper",
    x=0,
    y=-0.1,
    bgcolor="white",
    font=dict(size=12, color="black"),  # 调整为您希望的大小
)


# 显示图表
fig.show()

C:\Users\redhawc\AppData\Local\Temp\ipykernel_21056\320219430.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\redhawc\AppData\Local\Temp\ipykernel_21056\320219430.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\redhawc\AppData\Local\Temp\ipykernel_21056\320219430.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

### Mini-report

As a global financial centre, the UK, especially London—the 'City of Finance,' is renowned for its banking services. This led to my business question: Which UK city is the most suitable for establishing a new bank?

To investigate this question, I utilized the Advanced search feature of the Companies House API to search for all businesses established since 2015 with the SIC code 64191 (the SIC code for Banks). This yielded precise search results, which were used to create an interactive, year-by-year scatter bubble plot map.

The visualization shows that, before 2020, major UK cities including Cardiff and Leeds maintained a high number of new bank establishments and a low rate of bank closures each year, reflecting a relative prosperity in the banking sector. However, possibly influenced by Covid-19 in 2020, all cities except London experienced a reduction in bubble size and a darkening of their colour, indicating a significant decrease in new bank establishments and a surge in bank closure rates. Particularly in Edinburgh, there were no new banks established. However, London maintained a relatively low bank closure rate and even saw an increase in new bank establishments. This might be attributed to the scale effect of London's banking industry, offering a kind of 'strength in numbers' for new banks during market downturns. In the subsequent years of 2021-2022, the bubbles representing all cities were noticeably darker and smaller than those before 2020, representing the severe situation in the banking sector. Yet, London continued to display its vitality as a financial hub with its larger and lighter-coloured bubble. Undoubtedly, London is the most suitable city in the UK for establishing a new bank.


